In [1]:
import csv 

# Load the required libraries
import pandas as pd
from googlemaps import Client as GoogleMaps
import googlemaps
import gmaps
from tqdm import tqdm

from googleplaces import GooglePlaces, types, lang

from datetime import datetime

import json

import geopandas
import geopy

import csv

import pandas as pd

from tqdm import tqdm

import jsonlines
fname = 'geocode.jsonl'
        
import json


def save_csv(path,columns,tpl):
        
    with open(path,'w') as out:
        csv_out=csv.writer(out)
        csv_out.writerow(columns)
        csv_out.writerow(tpl)
        
def update_csv(path,tpl):
    with open(path,'a') as out:
        csv_out=csv.writer(out)
        csv_out.writerow(tpl)

In [2]:
vic_df = pd.read_csv('vicgeocoded.csv');

In [3]:
train_df = pd.read_csv('nearest_train.csv');
train_df = train_df.rename(columns={'train_duration': 'train_duration_val', 'train_duration.1': 'train_duration_unit'})

In [4]:
train_df.columns

Index(['address', 'lat', 'lng', 'train_name', 'train_lat', 'train_lng',
       'train_distance_val', 'train_distance_unit', 'train_duration_val',
       'train_duration_unit'],
      dtype='object')

In [5]:
left_merged = pd.merge(vic_df, train_df, how="left", on=["address"]);
left_merged = left_merged[left_merged['train_name'].isnull() ==False];

In [6]:
left_merged.train_distance_unit.unique()

array(['km', 'm'], dtype=object)

In [7]:
left_merged.train_duration_unit.unique()

array(['mins', 'min', 'hour', 'hours'], dtype=object)

In [8]:
def to_m(val,unit):
    
    if unit == 'm':
        return val;
    if unit == 'km':
        return val*1000;
    
def to_s(val,unit):
    
    if unit == 'min' or unit == 'mins':
        return val;
    if unit == 'hour' or unit == 'hours':
        return val*60;

In [9]:
left_merged['train_distance_m'] = left_merged.apply(lambda row: to_m(row.train_distance_val,row.train_distance_unit), axis= 1);
left_merged['train_duration_min'] = left_merged.apply(lambda row: to_s(row.train_duration_val,row.train_duration_unit), axis= 1)

In [10]:
left_merged.columns

Index(['address', 'year', 'type_1', 'code', 'suburb', 'car', 'bath', 'bed',
       'rent_pw', 'rent_yminthree', 'latitude', 'longitude', 'lat', 'lng',
       'train_name', 'train_lat', 'train_lng', 'train_distance_val',
       'train_distance_unit', 'train_duration_val', 'train_duration_unit',
       'train_distance_m', 'train_duration_min'],
      dtype='object')

In [11]:
left_merged['prev_year'] = left_merged['year']-3;

In [12]:
model_df = left_merged[['address', 'year', 'prev_year', 'type_1', 'suburb', 'car', 'bath', 'bed',\
                        'train_duration_min','train_distance_m','rent_yminthree','rent_pw']];

In [13]:
model_df.to_csv('final.csv')

In [14]:
model_df

,address,year,prev_year,type_1,suburb,car,bath,bed,train_duration_min,train_distance_m,rent_yminthree,rent_pw
0,"/ 71 SOUTH AVENUE , ALTONA MEADOWS",2014,2011,house,Altona Meadows,1.0,1,2,6.0,3000.0,260.0,270.0
1,"0 ELLIS TRACK , GLADYSDALE",2015,2012,house,Gladysdale,1.0,1,1,44.0,39800.0,210.0,240.0
2,"0 ELLIS TRACK , GLADYSDALE",2015,2012,house,Gladysdale,1.0,1,1,44.0,39800.0,210.0,240.0
4,"001 / 2 YARRA BING CRES , BURWOOD",2014,2011,unit/apmt,Burwood,1.0,1,2,8.0,4900.0,430.0,410.0
5,"001 / 2 YARRA BING CRES , BURWOOD",2014,2011,unit/apmt,Burwood,1.0,1,2,8.0,4900.0,430.0,410.0
...,...,...,...,...,...,...,...,...,...,...,...,...
192817,"WELLINGTON PARADE , EAST MELBOURNE",2015,2012,unknown,East Melbourne,1.0,1,1,2.0,700.0,370.0,375.0
192818,"WELLINGTON PARADE , EAST MELBOURNE",2018,2015,unknown,East Melbourne,1.0,1,1,2.0,700.0,375.0,437.5
192819,"WELLINGTON PARADE , EAST MELBOURNE",2018,2015,unknown,East Melbourne,1.0,1,1,2.0,700.0,375.0,437.5
192824,"Y102 / 125 TURNER STREET , ABBOTSFORD",2020,2017,unknown,Abbotsford,1.0,1,1,6.0,2300.0,395.0,395.0


In [15]:
model_df.to_csv('final.csv',index=False)

In [92]:
model_df

,address,year,prev_year,type_1,suburb,car,bath,bed,train_duration_min,train_distance_m,rent_yminthree,rent_pw
0,"/ 71 SOUTH AVENUE , ALTONA MEADOWS",2014,2011,house,Altona Meadows,1.0,1,2,6.0,3000.0,260.0,270.0
1,"0 ELLIS TRACK , GLADYSDALE",2015,2012,house,Gladysdale,1.0,1,1,44.0,39800.0,210.0,240.0
2,"0 ELLIS TRACK , GLADYSDALE",2015,2012,house,Gladysdale,1.0,1,1,44.0,39800.0,210.0,240.0
27,"06 / 11 HAINES STREET , NORTH MELBOURNE",2017,2014,unit/apmt,North Melbourne,1.0,1,1,5.0,1500.0,290.0,300.0
28,"06 / 11 HAINES STREET , NORTH MELBOURNE",2017,2014,unit/apmt,North Melbourne,1.0,1,1,5.0,1600.0,290.0,300.0
...,...,...,...,...,...,...,...,...,...,...,...,...
170659,"UNIT 7 / 38 WORDSWORTH AVENUE , CLAYTON SOUTH",2022,2019,unknown,Clayton South,1.0,1,2,3.0,1600.0,365.0,380.0
170660,"UNIT 7 / 38 WORDSWORTH AVENUE , CLAYTON SOUTH",2022,2019,unknown,Clayton South,1.0,1,2,3.0,1600.0,365.0,380.0
170692,"UNIT 9 / 64 PARKSIDE AVENUE , KEILOR EAST",2019,2016,unknown,Keilor East,1.0,1,2,10.0,6800.0,325.0,350.0
170693,"UNIT 9 / 64 PARKSIDE AVENUE , KEILOR EAST",2022,2019,unknown,Keilor East,1.0,1,2,10.0,6800.0,350.0,380.0
